In [9]:
import csv
import json
import os
import requests
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('../..') / '.env'
load_dotenv(dotenv_path=env_path)

ISCRAPER_API_KEY = os.environ.get('ISCRAPER_API_KEY')
PROFILE_DETAILS_URL = "https://api.iscraper.io/v2/profile-details"
        
        

In [10]:
def call_iscraper(linkedin_id: str):
    payload = json.dumps(
        {
            "profile_id": linkedin_id,
            "profile_type": "personal",
            "network_info": True,
        }
    )
    headers = {"X-API-KEY": ISCRAPER_API_KEY, "Content-Type": "application/json"}

    response = requests.request(
        "POST", PROFILE_DETAILS_URL, headers=headers, data=payload
    )
    data = json.loads(response.text)

    return data

In [24]:
with open('ramp.csv', 'r') as rf:
    reader = csv.reader(rf, delimiter=',')
    column_names = next(reader)
    # print(column_names)

    linkedin_url_index = column_names.index('CONTACT_LINKEDIN_PROFILE_URL')
    connection_count_index = column_names.index('Connection count')
    linkedin_bio_index = column_names.index('Linkedin Bio?')

    with open('written_ramp.csv', 'w') as wf:
        writer = csv.writer(wf, delimiter=',')
        writer.writerow(['CONTACT_LINKEDIN_PROFILE_URL', 'Connection Count', 'Follower Count', 'LinkedIn Bio'])

        for row in reader:
            linkedin_url = row[linkedin_url_index]
            # print(linkedin_url)
            linkedin_id = linkedin_url.strip('/').split('/')[-1]
            if linkedin_id == "":
                continue
            # print(linkedin_id)

            data = call_iscraper(linkedin_id)
            if 'network_info' not in data:
                connections_count = -1
                followers_count = -1
            else:
                if 'connections_count' not in data['network_info']:
                    connections_count = -1
                else:
                    connections_count = data['network_info']['connections_count']
                if 'followers_count' not in data['network_info']:
                    followers_count = -1
                else:
                    followers_count = data['network_info']['followers_count']

            if 'summary' not in data:
                summary = ''
            else:
                summary = data['summary']
            
            writer.writerow([linkedin_url, connections_count, followers_count, summary])
            print(f'Wrote {linkedin_url}')


Wrote https://www.linkedin.com/in/thomas-kutschke-6b44319a
Wrote https://www.linkedin.com/sales/people/ACwAAAk3mLYBUtRCWNxeEOnM9Om0Bk1EhfEckBM
Wrote https://www.linkedin.com/in/hunter-mezzetti
Wrote https://www.linkedin.com/in/jon-holstrom-0654224
Wrote https://www.linkedin.com/in/jtroemel
Wrote https://www.linkedin.com/sales/people/ACwAAAHJNYUBNJiA0hR4f0tq7AfYq0HLjepb1Jk
Wrote https://www.linkedin.com/in/kevin-ondrus-55166085/
Wrote https://www.linkedin.com/in/kristina-aspirany-mba-a5239815b/
Wrote https://www.linkedin.com/in/scott-goebels-3b246369
Wrote https://www.linkedin.com/sales/people/ACwAACaWGR8BCqKEWsXVBX-gx7eOeW9m6-yHZks
Wrote https://www.linkedin.com/sales/people/ACwAAB0jaQ4BmgjF4lHccT9lhhUyjzuGHGvx3qo
Wrote https://www.linkedin.com/in/ashwin-prakash-cpa-19bb1541
Wrote https://www.linkedin.com/in/kory-kent-cpa-57416950
Wrote https://www.linkedin.com/in/jacob-miller-b8291b37
Wrote https://www.linkedin.com/sales/people/ACwAABchm34BSKji3vVbYKkfFSkE1-uZOEQqfog
Wrote https://www

KeyError: 'summary'